In [1]:
import IPython.display as ipd
import json
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import os

from dataloaders import ImageCaptionDataset, ImageCaptionDatasetHDF5
from run_unit_analysis import (load_dataset, get_word_ali, get_code_ali,
                               prepare_data, STOP_WORDS, comp_code_to_wordprec,
                               comp_word_to_coderecall)
from run_utils import load_audio_model_and_state
from steps.plot import (load_raw_spectrogram, plot_spec_and_alis,
                        plot_precision_recall, plot_num_words_above_f1_threshold)
from steps.unit_analysis import (print_code_to_word_prec, print_word_by_code_recall,
                                 comp_code_word_f1, print_code_stats_by_f1, 
                                 print_word_stats_by_f1, count_high_f1_words,
                                 compute_topk_avg_f1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# load data
# json_path = './filelist/train_original_with_alignments.json'
# hdf5_path = '/data/sls/temp/wnhsu/data/mm/places_hdf5/PlacesEnglish400kTrain.json'
# json_path = "/home/harwath/data/PlacesAudio_400k_distro/metadata/"
# hdf5_path = '/home/harwath/data/PlacesAudio_400k_distro/metadata/PlacesEnglish400kTrain.json'
MYTHESISHOME = "/home/crabtree/theses/ResDAVEnet-VQ/"
MYDATAHOME = os.path.join(MYTHESISHOME, "data/TrilingualData")
MYHDF5DATAHOME = os.path.join(MYDATAHOME, "hdf5/")
json_path = os.path.join(MYDATAHOME,"/metadata/")
hdf5_path = os.path.join(json_path, 'PlacesEnglish400kTrain.json')

# This is an alternative to ImageCaptionDatasetHDF5, which computes audio
# features on the fly. It might be much slower due to intense data I/O.
# dataset = ImageCaptionDataset(json_path)

dataset = ImageCaptionDatasetHDF5(hdf5_path)
with open(json_path) as f:
    json_data_basis = json.load(f)
    json_data = json_data_basis['data']
    audio_base = json_data_basis.get('audio_base_path', '')
    image_base = json_data_basis.get('image_base_path', '')

['get-pip.py', 'data', 'venv_thesis', 'ResDAVEnet-VQ']

### Get Image Dataset

In [3]:
import json
# in the metadata folder there is train.json and valid.json (and other but I don't know much about them yet)
image_json_path = "/home/harwath/data/PlacesAudio_400k_distro/metadata/train.json"
with open(image_json_path) as f:
    data = json.load(f)
# image_dataset = ImageCaptionDatasetHDF5(image_json_path)
data['data'][:2]

[{'uttid': 'A239X6YVFBE51-GSUN_D1B978F9E68240510A63EFD734FB650E',
  'speaker': 'A239X6YVFBE51',
  'asr_text': '3 happy travelers are posing for picture inside of a airport',
  'wav': 'wavs/1/utterance_333455.wav',
  'image': 'a/airport_terminal/gsun_d1b978f9e68240510a63efd734fb650e.jpg'},
 {'uttid': 'A2LVPGEPC1EUOW-GSUN_B2C654D7066748E3274693F8A870C799',
  'speaker': 'A2LVPGEPC1EUOW',
  'asr_text': 'picture is a bridge runs along a body of water to the right is a tree with no leaves on a hill',
  'wav': 'wavs/2/utterance_232878.wav',
  'image': 'v/viaduct/gsun_b2c654d7066748e3274693f8a870c799.jpg'}]

In [4]:
import h5py



# image_dataset = ImageCaptionDatasetHDF5(image_json_path)